<a href="https://colab.research.google.com/github/analyticsbyted/nfl/blob/main/nfl_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>